In [1]:
!pip3 install newspaper3k


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=845cbee6fadbdbe6a7d318982400d9d54a997ccd61797df7497b04e964568305
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=51aa21a0990ecca63195a03d05630119ef8ad1496d73ac63e9d990e96c16d1d2
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bd

In [2]:
import newspaper
import pandas as pd


In [ ]:
!pip install wget




In [6]:
import urllib
import os


for i in range(len(df['image_links'])):
    sp1=df['image_links'][i].split("['")[1]
    sp2=sp1.split("']")[0]
    image_url = sp2
    filename = f"img/image_{i}.jpg"  
    
    os.makedirs(os.path.dirname(filename), exist_ok=True)  
    
    urllib.request.urlretrieve(image_url, filename)

In [3]:
pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=fb1e23c6ea3bb946284eabf2338b76f617386164280e5aee2d1d8ac560fdb566
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [4]:
import newspaper
import pandas as pd
import requests
from langdetect import detect
import time
from newspaper.article import ArticleException

def scrape_articles(url, target_date, max_retries=3):
    try:
        page_features = newspaper.build(url, language='en', memoize_articles=False)
    except Exception as e:
        print(f"Failed to build newspaper object: {str(e)}")
        return None

    articles_data = []

    for article in page_features.articles:
        retry_count = 0
        while retry_count < max_retries:
            try:
                article.download()
                article.parse()
                article.nlp()
                
                
                if len(article.text) < 10:
                    print(f"Skipping article with insufficient text: {article.url}")
                    break
                
                article_language = detect(article.text)
                if article_language == 'en':
                    if (
                        article.publish_date
                        and article.publish_date.date() > pd.to_datetime(target_date).date()
                    ):
                        image_links = []
                        if article.top_image:
                            image_links.append(article.top_image)
                        videos = article.movies if article.movies else ["No videos found"]
                        articles_data.append({
                            'title': article.title,
                            'body': article.text,
                            'url': article.url,
                            'image_links': image_links,
                            'publish_date': article.publish_date.strftime('%Y-%m-%d'),
                            'videos': videos,
                            'summary': article.summary,
                            'keywords': article.keywords
                        })
                    else:
                        print(f"Skipping older article: {article.url}")
                    break  
                else:
                    print(f"Skipping non-English article: {article.url}")
                    break
            except ArticleException as ae:
                print(f"Failed to process article: {str(ae)}")
                retry_count += 1
                time.sleep(2)  # Wait for a short time before retrying
                continue
            except Exception as e:
                print(f"Unhandled exception: {str(e)}")
                break  

    if not articles_data:
        print("No articles found matching the criteria.")
        return None

    articles_df = pd.DataFrame.from_dict(articles_data)
    return articles_df

url = 'https://www.cnn.com/'
target_date = '2022-01-16'
result_df = scrape_articles(url, target_date)

if result_df is not None:
    result_df.to_csv('news02.csv', index=False)
else:
    print("No data to save.")


Failed to process article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/latest-headlines?mod=wsjheader on URL https://www.wsj.com/news/latest-headlines?mod=wsjheader
Failed to process article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/latest-headlines?mod=wsjheader on URL https://www.wsj.com/news/latest-headlines?mod=wsjheader
Failed to process article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/latest-headlines?mod=wsjheader on URL https://www.wsj.com/news/latest-headlines?mod=wsjheader
Failed to process article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/latest-headlines?mod=nav_top_section on URL https://www.wsj.com/news/latest-headlines?mod=nav_top_section
Failed to process article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/la